Reference:

[1] Frérot, Lucas Henri Galilée. ‘Bridging Scales in Wear Modeling with Volume Integral Methods for Elastic-Plastic Contact’, n.d.

[2] Hu, Y.Z. and Tonder, K., 1992. Simulation of 3-D random rough surface by 2-D digital filter and Fourier analysis. International journal of machine tools and manufacture, 32(1-2), pp.83-90. DOI: 10.1016/0890-6955(92)90064-N